# Two pipelines Problem

Cristina Hernández, Beatriz Jimenez, Macarena Vargas, Guillermo Ruiz

In [1]:
import pyomo.environ as pe
import pyomo.opt as po
from pyomo.environ import NonNegativeReals, Binary

#### Create the model 

In [2]:
model = pe.ConcreteModel()

#### Sets


${N}$ : Network nodes = {A, B, C, D, E, F, G, P1, P2}

In [3]:
model.N = pe.Set(initialize = ["A", "B", "C", "D", "E", "F", "G", "P1", "P2"])

${R}$ : Subset of Refineries  = {A, B, C, D, E, F, G}

In [4]:
model.R = pe.Set(initialize = ["A", "B", "C", "D", "E", "F", "G"])


${P}$ : Subset of Pipelines = {P1, P2}

In [5]:
model.P = pe.Set(initialize = ["P1", "P2"])


${E_{i, j}}$ : existing connection between refinery i and refinery j

In [6]:
E_undirected = [
    ("A","B"), ("B","C"), ("C","D"), ("D","E"), ("E","F"), ("F","A"),
    ("A","G"), ("B","G"), ("C","G"), ("D","G"), ("E","G"), ("F","G")
]

# Hacemos E bidireccional
E_bidir = set()
for (i,j) in E_undirected:
    E_bidir.add((i,j))
    E_bidir.add((j,i))          # arco inverso con misma capacidad

model.E = pe.Set(dimen=2, initialize=E_bidir)

${r_i}$ : Refineries you can reach from the refinery i

In [7]:
def outgoing_init(model, i):  # r(i)
    return [j for (ii,j) in model.E if ii == i]

model.r      = pe.Set(model.R, initialize=outgoing_init)

${r'_i}$ : Refineries that can reach the refinery i

In [8]:
def incoming_init(model, i):  # r'(i)
    return [ii for (ii,j) in model.E if j == i]

model.rprime = pe.Set(model.R, initialize=incoming_init)

#### Parameters

${Q_{i, j}}$ : available capacity between refinery i and refinery j [l/s]

In [9]:
Q_data = {
    ("A","B"): 600, ("B","C"): 700, ("C","D"): 700, ("D","E"): 650,
    ("E","F"): 550, ("F","A"): 700, ("A","G"): 850, ("B","G"): 950, 
    ("C","G"): 600, ("D","G"): 550, ("E","G"): 700, ("F","G"): 650
}

Q_bidir = {}
for (i,j), cap in Q_data.items():
    Q_bidir[(i,j)] = cap
    Q_bidir[(j,i)] = cap

model.Q = pe.Param(model.E, initialize=Q_bidir)

${D_{r}}$ : contracted capacity at refinery r [l/s]

In [10]:
demand_data = {"A": 700, "B": 650, "C": 450, "D": 570, "E": 490, "F": 630, "G": 810}

model.D = pe.Param(model.R, initialize = demand_data)

${PD_{r}}$ :  unmet demand penalty at refinery r [€/l/s]

In [11]:
penalty_data = {"A": 300, "B": 300 , "C": 400, "D": 300, "E": 450, "F": 340, "G": 350}

model.PD = pe.Param(model.R, initialize = penalty_data)

${S{p}}$ : supply capacity of new pipeline p [l/s]

In [12]:
supply_data = {"P1": 1500, "P2": 2500}

model.S = pe.Param(model.P, initialize = supply_data)

#### Variables

${X_{i, j}}$ : daily volume of oil from refinery i to refinery j [l/s]


In [13]:
PR = {(p, r) for p in model.P for r in model.R}
model.X_index = pe.Set(dimen=2, initialize=set(model.E.data()) | PR)
model.x = pe.Var(model.X_index, domain=NonNegativeReals)

${nsd_r}$ : non-served demand at refinery r [l/s]

In [14]:
model.nsd = pe.Var(model.R, domain = NonNegativeReals)

${u_{p, r}}$ : active connection from pipeline p to refinery r {0,1}

In [15]:
model.u = pe.Var(model.P, model.R, domain = Binary)

#### Objective Function

##### Minimize shortage of contracted demand [€] --> min $\sum_{r∈ℝ} PD_{r} * nsd_{r}$

In [16]:
def obj_rule(model):
    return sum(model.PD[r] * model.nsd[r] for r in model.R)

model.OBJ = pe.Objective(rule = obj_rule, sense = pe.minimize)

#### Constrains 

Constraint #1: Balance in refinery r [m3]

$\sum_{i∈e(i,r)} X_{i,r} - \sum_{i∈e(r,i)}  X_{r,i} = D_r - nsd_r \forall r$ 

In [17]:
def balance_rule(model, r):
    inflow_net  = sum(model.x[i, r] for i in model.rprime[r] if (i, r) in model.X_index)
    inflow_p    = sum(model.x[p, r] for p in model.P          if (p, r) in model.X_index)
    outflow_net = sum(model.x[r, j] for j in model.r[r]       if (r, j) in model.X_index)
    return inflow_net + inflow_p - outflow_net == model.D[r] - model.nsd[r]
model.Balance = pe.Constraint(model.R, rule=balance_rule)

Constraint #2: Maximum volume from refinery in i to refinery in j [l/s]

$X_{r,r'} \leq Q_{r,r'} \forall r, r' ∈ e(r, r')$ 

In [18]:
def capacity_rule(model, i, j):
    return model.x[i, j] <= model.Q[i, j]
model.Capacity = pe.Constraint(model.E, rule=capacity_rule)

Constraint #3: Maximum supplied volume from pipeline p to refinery r [l/s]

$X_{p,r} \leq S_p * u_{p,r} \forall p,r $ 

In [19]:
def pipeline_supply_rule(model, p, r):
    return model.x[p, r] <= model.S[p] * model.u[p, r]

model.PipelineSupply = pe.Constraint(model.P, model.R, rule=pipeline_supply_rule)

Constraint #4: Each pipeline p connects to a single refinery [l/s]

$\sum_{r∈e(p,r)} u_{p,r} \leq 1 \forall p$ 

In [20]:
def one_connection_rule(model, p):
    return sum(model.u[p, r] for r in model.R) == 1
model.OneConnection = pe.Constraint(model.P, rule=one_connection_rule)

Constraint #5: Both pipelines cannot connect adjacent nodes to refinery r

$ u_{p=1,r} + \sum_{r'∈e(r,r')} u_{p=2,r'} \leq 1 \forall r$ 

$ u_{p=2,r} + \sum_{r'∈e(r,r')} u_{p=1,r'} \leq 1 \forall r$ 

In [21]:
def no_adjacent_rule(model, r):
    return model.u["P1", r] + sum(model.u["P2", j] for j in model.r[r]) <= 1
model.NoAdjacentP1 = pe.Constraint(model.R, rule=no_adjacent_rule)

def no_adjacent_rule2(model, r):
    return model.u["P2", r] + sum(model.u["P1", j] for j in model.r[r]) <= 1
model.NoAdjacentP2 = pe.Constraint(model.R, rule=no_adjacent_rule2)

Constraint #6: Maximum non-served demand at refinery r [l/s]

$ nsd_r \leq D_r \forall r $

In [22]:
def max_unserved_rule(model, r):
    return model.nsd[r] <= model.D[r]
model.MaxUnserved = pe.Constraint(model.R, rule=max_unserved_rule)

#### Solving the problem  

In [23]:
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True)

print("Status:", result.solver.status)
print("Termination:", result.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2707272
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\crist\AppData\Local\Temp\tmpj7q_ya52.pyomo.lp
Reading time = 0.00 seconds
x1: 68 rows, 59 columns, 204 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 68 rows, 59 columns and 204 nonzeros
Model fingerprint: 0x33845fac
Variable types: 45 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [3e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 33 rows and 13 columns
Presolve time: 0.00s
Presolved: 35 rows, 46 columns, 135 nonzeros
Variable types: 33 continuous, 13 integer (13 binary)
Found heur

In [24]:
# === Pipeline connections ===
print("\nPipeline connections (u[p,r] = 1):")

try:
    x_index = set(model.X_index)
except Exception:
    x_index = set(model.x.index_set())

for p in model.P:
    connected = [r for r in model.R if pe.value(model.u[p, r]) > 0.5]
    if connected:
        r = connected[0]  # debería haber exactamente una por el constraint
        flow = pe.value(model.x[p, r]) if (p, r) in x_index else 0.0
        print(f"  {p} is connected to refinery {r}")
        print(f"  Flow from {p} to {r}: {flow:.1f} l/s")
    else:
        print(f"  {p} has no active connection")

# === Refinery status ===
print("\nRefinery status:")
total_nsd, total_cost = 0.0, 0.0
for r in model.R:
    D   = pe.value(model.D[r])
    nsd = pe.value(model.nsd[r])
    served = D - nsd
    cost_r = pe.value(model.PD[r]) * nsd
    total_nsd  += nsd
    total_cost += cost_r
    print(f"  {r}: demand={D:.0f}, served={served:.0f}, nsd={nsd:.0f}, penalty={cost_r:.0f} €")

print(f"\nTOTAL NSD = {total_nsd:.0f} l/s")
print("Economic impact (objective value) [€]:", pe.value(model.OBJ))

# === Significant flows in the network ===
print("\nSignificant flows in the network:")

# orden de fuentes: primero refinerías A..G, luego P1 y P2 al final
order_src = ['A','B','C','D','E','F','G','P1','P2']
order_tgt = ['A','B','C','D','E','F','G']  # solo refinerías como destino (los pipelines no reciben)

rank_src = {n:i for i,n in enumerate(order_src)}
rank_tgt = {n:i for i,n in enumerate(order_tgt)}

flows = []
TH = 1.0  # umbral para mostrar
# si tienes model.X_index úsalo; si no, recorre model.x directamente
idx = list(model.X_index) if hasattr(model, "X_index") else list(model.x)

for (i, j) in idx:
    v = pe.value(model.x[i, j])
    if v is not None and v >= TH:
        s_rank = rank_src.get(i, 999)
        t_rank = rank_tgt.get(j, 999)
        flows.append((s_rank, t_rank, i, j, v))

# ordenar por fuente (según orden deseado), luego por destino (A..G)
flows.sort(key=lambda t: (t[0], t[1], t[2], t[3]))

for _, __, i, j, v in flows:
    print(f"{i} -> {j}: {v:.1f} l/s")


Pipeline connections (u[p,r] = 1):
  P1 is connected to refinery D
  Flow from P1 to D: 1500.0 l/s
  P2 is connected to refinery A
  Flow from P2 to A: 2500.0 l/s

Refinery status:
  A: demand=700, served=700, nsd=0, penalty=0 €
  B: demand=650, served=350, nsd=300, penalty=90000 €
  C: demand=450, served=450, nsd=0, penalty=0 €
  D: demand=570, served=570, nsd=0, penalty=0 €
  E: demand=490, served=490, nsd=0, penalty=0 €
  F: demand=630, served=630, nsd=0, penalty=0 €
  G: demand=810, served=810, nsd=0, penalty=0 €

TOTAL NSD = 300 l/s
Economic impact (objective value) [€]: 90000.0

Significant flows in the network:
A -> B: 600.0 l/s
A -> F: 700.0 l/s
A -> G: 850.0 l/s
B -> C: 700.0 l/s
C -> B: 350.0 l/s
C -> G: 600.0 l/s
D -> C: 700.0 l/s
D -> E: 640.0 l/s
D -> G: 140.0 l/s
E -> G: 700.0 l/s
F -> A: 170.0 l/s
F -> E: 550.0 l/s
G -> A: 180.0 l/s
G -> B: 100.0 l/s
G -> D: 550.0 l/s
G -> F: 650.0 l/s
P1 -> D: 1500.0 l/s
P2 -> A: 2500.0 l/s
